In [1]:
import time

import numpy as np
# import torch

import tools.rotations as rot
# import pytorch_kinematics as pk

from controller import locomotion
from control_interface_real_robot import real_robot
import mujoco
import cvxpy as cp
from control_interface_real_robot import q_sim2real

import copy
%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'rclpy'

In [2]:
import sys
sys.executable

'/usr/bin/python3'

In [3]:
hand_real = real_robot(control_mode='position', legs=5)

leg num: 5


In [4]:
hand_real.update_data()
hand_real.q

array([ 0.32980588,  0.20248547, -0.4878059 ,  0.01994175,  0.15953401,
       -0.13499032,  0.15493207,  0.23623304, -0.28225246, -0.42644668,
        0.18100974, -0.21629129,  0.05522331,  0.02914564, -0.04141748])

In [5]:
q0 = np.array([-1.62289571e-04, -3.50667052e-04,  7.98538815e-02,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
               -0.7,   7.91144283e-01,  0.3,  0.3,
               0,  7.93214611e-01,  0.3,  0.3,  0.00000000e+00,
               -1.47079633e+00, -8.00000000e-01, -8.00000000e-01, 
               -6.26414686e-06,  7.93034999e-01,  0.3,  0.3,
               -1.00000000e+00,  7.92384667e-01,  0.3,  0.3])
q0.shape

(27,)

In [2]:
# test pytorch kinematics
xml_path = 'descriptions/five_finger_hand_ssss.xml'
# chain = pk.build_chain_from_mjcf('descriptions/single_finger_ssss.xml', 'finger_1')
# print(chain)
q0 = np.array([-1.62289571e-04, -3.50667052e-04,  7.98538815e-02,  9.99999738e-01,
       -4.46340938e-04,  5.67392530e-04, -4.89152197e-05, 
               0,   7.91144283e-01,  0.3,  0.3,
               0,   7.91144283e-01,  0.3,  0.3, 
               0,   7.91144283e-01,  0.3,  0.3, 
               0,   7.91144283e-01,  0.3,  0.3,
               0,   7.91144283e-01,  0.3,  0.3,
               ])
r = locomotion(xml_path, q0=q0)

print('locomotion freq', r.f)
# take a rest to let the acceleration to zero
t0 = time.time()
q0 = np.copy(r.q)
while 1:
    # r.step()
    # r.send_torque(r.C)
    r.joint_computed_torque_control(q0)
    if time.time() - t0 > 2:
        break
import  copy
xh_local_init = copy.deepcopy(r.xh_local) 

Start to load xml file to build the kinematics. This might take about 20s, but only for the first time.
Kinematic model has been saved to kinematics/q2pose_5.txt
locomotion freq 1.0


NameError: name 'copy' is not defined

In [4]:
t0 = time.time()
q0 = np.copy(r.q)
while 1:
    # r.step()
    # r.send_torque(r.C)
    r.joint_computed_torque_control(q0)
    if time.time() - t0 > 2:
        break
import  copy
xh_local_init = copy.deepcopy(r.xh_local) 

In [ ]:
xh_local_init

In [ ]:
hand_real.q

In [ ]:
# q0 = np.array([ 0.13345633,   0.69029135,  0.41264084 , 
#                 -0.3,  0.69029135,  0.41264084,
#                 -0.00153398,   0.69029135,  0.41264084, 
#                 -0.11965051,  0.69029135,  0.41264084,
#                 1,   0.69029135,  0.41264084])
# 
# 
# # q_cmd = q_sim2real(q0, legs=5)
# hand_real.move_to_joints(q0.reshape(1,-1), sim2real=False)  # move to init pose

In [ ]:
hand_real.update_data()
hand_real.q

In [ ]:
# q_cmd = np.zeros(12)
# q_cmd[0] = 0.5
# hand_real.move_to_joints(q_cmd)

# q_cmd_sim = np.zeros(20)
# q_cmd_sim[13] = 0 
# q_cmd_sim[17] = 0
# r.modify_joint(q_cmd_sim)

# q_cmd = q_sim2real(q0, legs=5)
# 
# hand_real.move_to_joints(q_cmd, sim2real=False)  # move to init pose
# hand_real.send_position(q_cmd)
# hand_real.move_to_joints_by_vel(q_cmd)


In [344]:
xy_step

array([ 0.  , -0.01])

In [351]:
## gait design for four legs
 # this is the init position


t0 = time.time()
r.f = 2
T = 1 / r.f
v = np.array([0.0, -0.04,0]) 
# v = np.array([0.04,0 ,0]) 
# v = np.array([0, -0.02,0])
h = 0.05  # z axis lifting height for swing legs
r.vel_cmd[:3] = v

xy_step = v[:2] * T/2 * 0.5  # based the MIT paper
x_cmd = copy.deepcopy(xh_local_init)
x_cmd = [a[:3] for a in x_cmd]  # position command for legs
q_cmd_list = []
t_total = 10 * T

k = 0
q_desired = copy.deepcopy(r.q)
while 1:
    
    tau_Grf = np.zeros(20)
    t_now  = time.time() - t0
    t_period = t_now % (1/r.f)  # [0, T]
    
    if t_period < T/2: # FL and BR, front left and back right legs move during [0, T/2]
        swing_legs = [0, 2, 3]
        stance_legs = [1, 4]
        # swing_legs = [0, 1]
        # stance_legs = [3, 4]
    else:
        swing_legs = [1, 4]
        stance_legs = [0, 2, 3]
        # swing_legs = [3, 4]
        # stance_legs = [0, 1]
    
    # update vel direction in hand base frame
    # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
    # xy_step = xy_step[:2]
    for i in swing_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp/ (T/2), xh_local_init[i][:2] - xy_step, xh_local_init[i][:2] + xy_step)
        if t_tmp <= T/4:
            x_cmd[i][2] = r.bezier(t_tmp /(T/4), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
        else:
            x_cmd[i][2] = r.bezier((t_tmp - T/4) /(T/4), xh_local_init[i][2] + h, xh_local_init[i][2])
    
    for i in stance_legs:
        t_tmp = t_period % (T/2)
        x_cmd[i][:2] = r.bezier(t_tmp /(T/2), xh_local_init[i][:2] + xy_step, xh_local_init[i][:2] - xy_step)
        x_cmd[i][2] = xh_local_init[i][2]
    
    q_desired = r.hand_kine.ik(x_cmd, q_init=q_desired)
    r.joint_computed_torque_control(q_desired)
    # k += 1
    # q_desired[8:12] = q0[8:12]
    q_cmd = q_sim2real(q_desired, legs=5)
    q_cmd_list.append(q_cmd)
    # hand_real.send_position(q_cmd)

    if t_now > t_total:
        break

q_cmd_list = np.vstack(q_cmd_list)
q_cmd_list.shape

KeyboardInterrupt: 

In [352]:
def generate_traj(v=np.array([0.0, -0.04,0]), w=None, f = 2, num=1):
    """
    generate trajctory for moving forward or rotation    
    :param v: 
    :param w: 
    :param f: frequency
    :param num: number of period
    :return: 
    """    
    T = 1 / f
    # v = np.array([0.0, -0.04,0]) 
    # # v = np.array([0.04,0 ,0]) 
    # # v = np.array([0, -0.02,0])
    h = 0.05  # z axis lifting height for swing legs
    
    if w is None:
        xy_step = [v[:2] * T/2 * 0.5] * 5  # based the MIT paper
    else:
        theta = w * T/2 * 0.5
        R = np.array([[np.cos(theta), -np.sin(theta)], 
                      [np.sin(theta), np.cos(theta)]])
        xy_step = [xh_local_init[i][:2] @ R - xh_local_init[i][:2] for i in range(5)]
        
    x_cmd = copy.deepcopy(xh_local_init)
    x_cmd = [a[:3] for a in x_cmd]  # position command for legs
    q_desired = copy.deepcopy(q0)
    
    dt = 0.002
    q_len = int(num * T / dt)
    q_cmd_list = []
    for a in range(q_len):
        t_now  = a * dt
        t_period = t_now % (1/r.f)  # [0, T]
        
        if t_period < T/2: # FL and BR, front left and back right legs move during [0, T/2]
            swing_legs = [0, 2, 3]
            stance_legs = [1, 4]
            # swing_legs = [0, 1]
            # stance_legs = [3, 4]
        else:
            swing_legs = [1, 4]
            stance_legs = [0, 2, 3]
            # swing_legs = [3, 4]
            # stance_legs = [0, 1]
        
        # update vel direction in hand base frame
        # xy_step = rot.quat2mat(r.x[3:]).T @ np.array([xy_step[0], xy_step[1], 0])
        # xy_step = xy_step[:2]
        for i in swing_legs:
            t_tmp = t_period % (T/2)
            x_cmd[i][:2] = r.bezier(t_tmp/ (T/2), xh_local_init[i][:2] - xy_step[i], xh_local_init[i][:2] + xy_step[i])
            if t_tmp <= T/4:
                x_cmd[i][2] = r.bezier(t_tmp /(T/4), xh_local_init[i][2], xh_local_init[i][2] + h)  # z axis
            else:
                x_cmd[i][2] = r.bezier((t_tmp - T/4) /(T/4), xh_local_init[i][2] + h, xh_local_init[i][2])
        
        for i in stance_legs:
            t_tmp = t_period % (T/2)
            x_cmd[i][:2] = r.bezier(t_tmp /(T/2), xh_local_init[i][:2] + xy_step[i], xh_local_init[i][:2] - xy_step[i])
            x_cmd[i][2] = xh_local_init[i][2]
        
        q_desired = r.hand_kine.ik(x_cmd, q_init=q_desired)
        q_cmd = q_sim2real(q_desired, legs=5)
        q_cmd_list.append(q_cmd)
    q_cmd_list = np.vstack(q_cmd_list)
    print('q_cmd_list', q_cmd_list.shape)
    
    return q_cmd_list


In [353]:
q_cmd_list = generate_traj()

q_cmd_list (250, 15)


In [1138]:
# human hand shape
q11 = np.array([[1, 0,0,
                    0.5, 0,0,
                    0.,0,0,
                    -0.5,0,0,
                    -1,0,0]])
hand_real.move_to_joints(q11)   # 1-1




In [1139]:
# stand up
f = 3
num = 3
t_total = num/f
q_cmd_list = generate_traj(v=np.array([-0.04, 0,0]), f=2, num=num)  # generate traj

hand_real.move_to_joints(q_cmd_list[0,:].reshape(1,-1), sim2real=False)  # move to init pose

q_cmd_list (750, 15)


In [1143]:
# moving along straight line
f = 3
num = 6
t_total = num/f
q_cmd_list = generate_traj(v=np.array([-0.01, -0.03,0]), f=2, num=num)  # generate traj

hand_real.move_to_joints(q_cmd_list, t=t_total)  # send to the real robot
time.sleep(t_total)

q_cmd = q_sim2real(q0, legs=5)
hand_real.move_to_joints(q_cmd, sim2real=False, t=0.2)  # move to init pose

q_cmd_list (1500, 15)


In [1141]:
# rotation along z axis
f = 3
num = 1
t_total = num/f
q_cmd_list = generate_traj(w=0.2, f=2, num=num)

hand_real.move_to_joints(q_cmd_list, t=t_total)
time.sleep(t_total)

q_cmd = q_sim2real(q0, legs=5)
hand_real.move_to_joints(q_cmd, sim2real=False, t=0.2)  # move to init pose


q_cmd_list (500, 15)


In [398]:
# for grasping
# hand_real.move_to_joints(np.zeros([1,15]))   # all zeros



In [873]:
# first grasp
q11 = np.array([[0,0.4,0,
                    0,0.4,0,
                    0.2,0,0,
                    -0.2,0,0,
                    0,0.4,0]])
hand_real.move_to_joints(q11)   # 1-1


In [872]:
# first grasp
q12 = np.array([[0,0.7,0,
                    0,0.7,0,
                    0.8,0,0,
                    -0.8,0,0,
                    0,0.7,0]])
hand_real.move_to_joints(q12)   # 1-1

In [863]:
# first grasp
q13 = np.array([[0,0.7,0,
                    0,0.7,0,
                    0.8,-0.6,0,
                    -0.8,-0.6,0,
                    0,0.7,0]])
hand_real.move_to_joints(q13)   # 1-1

In [864]:
# first grasp
q13 = np.array([[0,0.7,0,
                    0,0.7,0,
                    1,-1,-0.5,
                    -1,-1,-0.5,
                    0,0.7,0]])
hand_real.move_to_joints(q13)   # 1-1

In [868]:
# first grasp
q13 = np.array([[0,0.7,0,
                    0,0.7,0,
                    0.5,-1.4,-1,
                    -0.5,-1.4,-1,
                    0,0.7,0]])
hand_real.move_to_joints(q13)   # 1-1

In [866]:
# # first grasp
# q13 = np.array([[0,0.7,0,
#                     0,0.7,0,
#                     -1,-1.4,-1,
#                     1,-1.4,-1,
#                     0,0.7,0]])
# hand_real.move_to_joints(q13)   # 1-1

In [1144]:
# second grasp
q13[0,6:12] = np.array([0.2,0,0,
                    -0.2,0,0,])
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                    0.2,0,0,
                    0.,0,0,
                    0,0.0,0]])

hand_real.move_to_joints(q13)   # 2-1



In [1125]:
# second grasp
q13[0,6:12] = np.array([0.2,0,0,
                    -0.2,0,0,])
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                    0.2,0,0,
                    0.,0,0,
                    0,0.0,0]])

hand_real.move_to_joints(q13)   # 2-1



In [1145]:
# second grasp
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.,0,0,
                    -0.5,0,0,
                    0,0.0,0]])
hand_real.move_to_joints(q13)   # 2-1

In [1146]:
# second grasp
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.8,0,0,
                    -0.8,0,0,
                    0,0.0,0]])

hand_real.move_to_joints(q13)   # 2-1
time.sleep(2.)

q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.8,-0.6,0,
                    -0.8,-0.6,0,
                    0,0.0,0]])
hand_real.move_to_joints(q13)   # 2-1
time.sleep(2.)

q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.8,-1.3,-0.5,
                    -0.8,-1.3,-0.5,
                    0,0.0,0]])
hand_real.move_to_joints(q13)   # 2-1
time.sleep(2.)




In [1147]:
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    0,0.0,0]])
hand_real.move_to_joints(q13)   # 2-1
time.sleep(2.1)

In [1148]:
q13 = np.array([[0,0.,0,
                    0,0.0,0,
                   0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    0,0.0,0]])
hand_real.move_to_joints(q13)   # 2-1
time.sleep(2.1)

# move back

In [1131]:
############## for moving back

q21 = np.array([[0,-0.5,0,
                    -1,-0.5,0,
                   0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    1,-0.5,0]])
hand_real.move_to_joints(q21)   # 2-1
time.sleep(2.1)




In [1132]:
############## for moving back

q21 = np.array([[0,-0.5,0,
                    -1,-0.5,0,
                 0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    1,-0.5,0]])
hand_real.move_to_joints(q21)   # 2-1
time.sleep(2.1)




In [1133]:
q22 = np.array([[0,0.5,0.5,
                    0, 0.5,0.5,
                 0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    0.5, 0.5,0.5]])
hand_real.move_to_joints(q22)   # 2-1
# time.sleep(2.1)


In [1134]:
q23 = np.array([[0,0.8,1.3,
                    1, 0.5,0.5,
                0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    0.5, 0.5,0.5]])
hand_real.move_to_joints(q23)   # 2-1
time.sleep(2.1)


In [1135]:
q24 = np.array([[0,-0.4,-0.5,
                    1, 0.5,0.5,
                0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
                    -1, 0.5,0.5]])
hand_real.move_to_joints(q24)   # 2-1
# time.sleep(2.1)

In [1136]:
q_back = np.vstack([q21,q22,q23,q24])

In [1150]:
hand_real.move_to_joints(q_back, t = 5) 

In [831]:
# move back
from control_interface_real_robot import q_sim2real


real_robot_loco = '1006.npz'
path = 'data_records/grasp_synthesis/real_hand/locomotion_data/'

d = np.load(path + real_robot_loco)  # fingers=fingers, d_list=d_list[q_choice], solution=solution, solution_fitness=solution_fitness
fingers = d['fingers']
solution = d['solution']

# 1006.npz with 3 objects
q_cmd_list = []
dt = 0.002
f = 0.3
nums = 1000 * 3
q_grasp = np.zeros(8)
# q_grasp = np.copy()    todo
def q2_real(q):
    assert len(q) == 4*3
    q_new = np.concatenate([q[4:12], q_grasp, q[:4]])
    q_cmd = q_sim2real(q_new, legs=5)
    return q_cmd

for i in range(nums):
    q = solution[:12] +  solution[12:24] * np.sin(2 * np.pi * f * i * dt +  solution[24:])  # (12,)
    q_cmd = q2_real(q)
    q_cmd[0, 6:12] = q13[0, 6:12]  # keep the two fingers static for grasping
    q_cmd_list.append(q_cmd)
q_cmd_list = np.vstack(q_cmd_list)


q_cmd_list.shape

(3000, 15)

In [157]:
q_cmd_list[0,:]

array([-0.22577383, -0.69147212,  0.25286284, -1.29950412,  1.85737641,
       -0.75406169, -0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.        ,  0.83783887, -0.33136557,  2.31181321])

In [235]:
hand_real.move_to_joints(q_cmd_list[0:1,:], t=1)  # swimming motion

In [236]:
hand_real.move_to_joints(q_cmd_list, t=nums*dt)

In [15]:
# q_align = np.array([0,np.pi/2,0,
#           0,np.pi/2,0,
#           0.00000000e+00, -1.47079633e+00, -8.00000000e-01, 
#                0.00000000e+00, -1.47079633e+00, -8.00000000e-01,
#            0,np.pi,0])
# 
# hand_real.move_to_joints(q_align.reshape(1,-1), sim2real=False, t=0.3)  # lie down for attactment

In [1151]:
q_align = np.array([0,np.pi/2,1,
          0,np.pi/2,1,
         0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
           0,np.pi,1])

hand_real.move_to_joints(q_align.reshape(1,-1), sim2real=False, t=0.3)  # 1, lie down for attactment



In [288]:
q_align = np.array([0,np.pi/2,0,
          0,np.pi/2,0,
         0.5,-1.4,-0.5,
                    -0.5,-1.4,-0.7,
           0,np.pi,0])

hand_real.move_to_joints(q_align.reshape(1,-1), sim2real=False, t=0.3)  # 2,  move up for better alignment

In [ ]:
and_real.move_to_joints(q_cmd_list[0:1,:], t=1)  # swimming motion
hand_real.move_to_joints(q_cmd_list, t=nums*dt)
